In [1]:
!pip install gradio

     |████████████████████████████████| 2.9 MB 12.2 MB/s 
     |████████████████████████████████| 212 kB 23.2 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 53 kB 507 kB/s 
     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 2.0 MB 51.4 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 253 kB 76.2 MB/s 
     |████████████████████████████████| 271 kB 52.1 MB/s 
     |████████████████████████████████| 144 kB 56.0 MB/s 
     |████████████████████████████████| 94 kB 835 kB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 10.9 MB 54.7 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 856 kB 49.8 MB/s 
     |████████████████████████████████| 4.0 MB 48.4 MB/s 
     |█████████████████████

In [2]:
!wget https://huggingface.co/spaces/yuhe6/final_project/resolve/main/Net2_Flip_jit.pt

--2022-05-11 05:12:54--  https://huggingface.co/spaces/yuhe6/final_project/resolve/main/Net2_Flip_jit.pt
Resolving huggingface.co (huggingface.co)... 34.228.99.153, 34.197.58.156, 34.225.34.242, ...
Connecting to huggingface.co (huggingface.co)|34.228.99.153|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/88/0a/880aba12dede51db32b45ee51b542cea9d10fb2dcf2a0816646ee2b694e8c0f6/f10dda32ac52651942a9142d9fed0e02c1ba5ce7e388241be0f3cfa56ebf289a [following]
--2022-05-11 05:12:54--  https://cdn-lfs.huggingface.co/repos/88/0a/880aba12dede51db32b45ee51b542cea9d10fb2dcf2a0816646ee2b694e8c0f6/f10dda32ac52651942a9142d9fed0e02c1ba5ce7e388241be0f3cfa56ebf289a
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 18.67.65.119, 18.67.65.85, 18.67.65.82, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|18.67.65.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220425179 (210M) [binary/

In [3]:
import os
import torch
from PIL import Image
from torchvision import transforms
import gradio as gr
#https://huggingface.co/spaces/yuhe6/final_project/blob/main/Net_Rotate9.pth
#os.system("wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt")

#model = torch.hub.load('huawei-noah/ghostnet', 'ghostnet_1x', pretrained=True)
#model = torch.jit.load('https://huggingface.co/spaces/yuhe6/final_project/blob/main/Net_Rotate9.pth').eval().to(device)


model = torch.jit.load('Net2_Flip_jit.pt', map_location = torch.device('cpu'))
model.eval()

model_categories = ["cat","dog"] # verify order
n_categories = len(model_categories)

#torch.hub.download_url_to_file('https://huggingface.co/spaces/yuhe6/final_project/blob/main/Net_Rotate9.pth', '/tmp/temporary_file')
#model = torch.hub.load('/tmp', 'temporary_file', pretrained=True)

#model.eval()
# Download an example image from the pytorch website
torch.hub.download_url_to_file("https://upload.wikimedia.org/wikipedia/commons/5/5b/Dog_%28Canis_lupus_familiaris%29_%281%29.jpg", "dog1.jpg")
torch.hub.download_url_to_file("https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Golde33443.jpg/640px-Golde33443.jpg", "dog2.jpg")
torch.hub.download_url_to_file("https://upload.wikimedia.org/wikipedia/commons/c/c7/Tabby_cat_with_blue_eyes-3336579.jpg", "cat1.jpg")
torch.hub.download_url_to_file("https://upload.wikimedia.org/wikipedia/commons/9/9e/Domestic_cat.jpg", "cat2.jpg")

def inference(input_image):
    preprocess = transforms.Compose([
        transforms.Resize(size = (256, 256)), # Fixed resize from transforms.Resize(256)
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Used print statements to detect shapes between input tensor & batch
    # e.g. input_tensor.shape
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch) # model(input_tensor) # needed to have batch dimension

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0])

    # Read the categories
    #with open("dog_cat.txt", "r") as f:
        #categories = [s.strip() for s in f.readlines()]
    #with open("dog_cat.txt", "r") as f:

    #categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top1_prob, top1_catid = torch.topk(probabilities, n_categories)
    result = {}
    for i in range(top1_prob.size(0)):
        result[model_categories[top1_catid[i]]] = top1_prob[i].item()
    return result

inputs = gr.inputs.Image(type='pil')
outputs = gr.outputs.Label(type="confidences", num_top_classes = n_categories)

title = "STAT 430 Final Project App -- Made by Group DHZ"
description = "This is our Cat & Dog Classifier for the final project, and the model we use is generated by our second neural network augmented by the flipping technique, which is would give the best accuracy. To use it, simply upload your image, or click one of the examples to load them. The authors are Xiongjie Dai (xdai12), Yu He (yuhe6), Mengjia Zeng (mengjia6)."
#article = "<p style='text-align: center'><a href='https://arxiv.org/abs/1911.11907'>GhostNet: More Features from Cheap Operations</a> | <a href='https://github.com/huawei-noah/CV-Backbones'>Github Repo</a></p>"

examples = [
  ['dog1.jpg'], 
  ['cat1.jpg'], 
  ['dog2.jpg'], 
  ['cat2.jpg']
]

gr.Interface(
  inference, inputs, outputs, 
  title = title, description = description, 
  examples = examples,
  analytics_enabled = False).launch(
    #debug = True # Enabled debug mode to see the stacktrace on Google Colab.
  )

/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


  0%|          | 0.00/216k [00:00<?, ?B/s]

  0%|          | 0.00/159k [00:00<?, ?B/s]

  0%|          | 0.00/2.02M [00:00<?, ?B/s]

  0%|          | 0.00/1.26M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://31967.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f0875f5c210>,
 'http://127.0.0.1:7860/',
 'https://31967.gradio.app')